# Akka.Streams Integration with Akka.NET Actors
Behind the scenes, all Akka.Streams graphs are ultimately compiled via the `IMaterializer` into one or more Akka.NET actors. Thus there are a number of ways Akka.Streams can integrate with Akka.NET actors in real-world use.

## Integration with Actors in Flows
Using techniques such as the `Ask<TResult>` method in Akka.NET, we can incorporate actors directly inside built-in Akka.Streams flows without much code.

In [ ]:
#r "nuget: Akka.Streams, 1.4.24"

using System.Linq;
using System.Collections.Immutable;
using Akka;
using Akka.Actor;
using Akka.Streams;
using Akka.Streams.Dsl;

ActorSystem actorSystem = ActorSystem.Create("StreamsExample");

IMaterializer materializer = actorSystem.Materializer();

// create a simple actor that performs some modulus
public class MyActor : ReceiveActor{
    public MyActor(){
        // returns a (i, i mod 2) ValueTuple
        Receive<int>(i => Sender.Tell((i, i % 2)));
    }
}

IActorRef myActorRef = actorSystem.ActorOf(Props.Create(() => new MyActor()), "myactor");

Installed Packages Akka.Streams, 1.4.24

In this instance, we've created a simple `ReceiveActor` that will perform a bit of modulus. Let's integrate it inside a simple graph and see how it works.

In [ ]:
// create 10,000 integers
Source<int,NotUsed> source = Source.From(Enumerable.Range(0, 10000));

// allow up to 10 parallel asks of our actor
var askFlow = Flow.Create<int>().SelectAsync(10, i => myActorRef.Ask<(int value, int mod)>(i, TimeSpan.FromSeconds(3)));

// create a flow that does the following:
// int --> is even number? --> keep only even numbers --> count total even numbers
var graph = source.Via(askFlow).Where(d => d.mod == 0).Select(x => x.value).To(Sink.ForEach<int>(i => Console.WriteLine(i)));

graph.Run(materializer);